采用resnet_v2_50来对绝缘子数据分类

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time

from datasets import dataset_utils

# Main slim library
from tensorflow.contrib import slim

### Download the resnet_v2_50 checkpoint

In [ ]:
url = "http://download.tensorflow.org/models/resnet_v2_50_2017_04_14.tar.gz"
checkpoints_dir = '/tmp/resnet_v2_50/checkpoints'

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

### Fine-tune the model on insulator dataset

In [2]:
from preprocessing import vgg_preprocessing

def load_batch(dataset, batch_size=16, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Resnet_v2_50, the same as vgg_net.
    image = vgg_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, images_raw, labels

In [4]:
import os
from datasets import insulators
from nets import resnet_v2

checkpoints_dir = '/tmp/resnet_v2_50/checkpoints'

image_size = resnet_v2.resnet_v2_50.default_image_size

insulators_data_dir = '/workspace/JiaXuejian'

def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    
    checkpoint_exclude_scopes=["resnet_v2_50/logits", "resnet_v2_50/SpatialSqueeze", "resnet_v2_50/predictions"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                break
        else:
            variables_to_restore.append(var)
    
    return slim.assign_from_checkpoint_fn(
      os.path.join(checkpoints_dir, 'resnet_v2_50.ckpt'),
      variables_to_restore)


train_dir = '/tmp/resnet_v2_50_finetuned/'

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = insulators.get_split('train', insulators_data_dir)
    images, _, labels = load_batch(dataset, height=image_size, width=image_size)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(resnet_v2.resnet_arg_scope()):
        logits, _ = resnet_v2.resnet_v2_50(images, num_classes=dataset.num_classes, is_training=True)
        
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=train_dir,
        init_fn=get_init_fn(),
        number_of_steps=2000)
        
  
print('Finished training. Last batch loss %f' % final_loss)

INFO:tensorflow:Summary name losses/Total Loss is illegal; using losses/Total_Loss instead.
INFO:tensorflow:Restoring parameters from /tmp/resnet_v2_50_finetuned/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /tmp/resnet_v2_50_finetuned/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global step 0: loss = 1.3017 (4.421 sec/step)
INFO:tensorflow:global step 1: loss = 4.3414 (0.887 sec/step)
INFO:tensorflow:global step 2: loss = 1.6124 (1.313 sec/step)
INFO:tensorflow:global step 3: loss = 1.2992 (1.157 sec/step)
INFO:tensorflow:global step 4: loss = 1.5188 (1.002 sec/step)
INFO:tensorflow:global step 5: loss = 1.7271 (0.900 sec/step)
INFO:tensorflow:global step 6: loss = 1.9158 (1.106 sec/step)
INFO:tensorflow:global step 7: loss = 13.6463 (1.242 sec/step)
INFO:tensorflow:g

INFO:tensorflow:global step 123: loss = 3.7477 (1.188 sec/step)
INFO:tensorflow:global step 124: loss = 3.8546 (0.981 sec/step)
INFO:tensorflow:global step 125: loss = 4.0477 (0.929 sec/step)
INFO:tensorflow:global step 126: loss = 3.8065 (0.969 sec/step)
INFO:tensorflow:global step 127: loss = 5.7967 (0.992 sec/step)
INFO:tensorflow:global step 128: loss = 4.1138 (0.927 sec/step)
INFO:tensorflow:global step 129: loss = 3.8695 (0.961 sec/step)
INFO:tensorflow:global step 130: loss = 4.8446 (1.011 sec/step)
INFO:tensorflow:global step 131: loss = 3.8924 (0.871 sec/step)
INFO:tensorflow:global step 132: loss = 3.7766 (1.456 sec/step)
INFO:tensorflow:global step 133: loss = 3.7837 (0.813 sec/step)
INFO:tensorflow:global step 134: loss = 3.7666 (1.032 sec/step)
INFO:tensorflow:global step 135: loss = 3.7695 (1.107 sec/step)
INFO:tensorflow:global step 136: loss = 3.7587 (0.807 sec/step)
INFO:tensorflow:global step 137: loss = 3.7386 (0.861 sec/step)
INFO:tensorflow:global step 138: loss = 

INFO:tensorflow:global step 252: loss = 2.9187 (0.897 sec/step)
INFO:tensorflow:global step 253: loss = 2.8875 (0.755 sec/step)
INFO:tensorflow:global step 254: loss = 2.9068 (1.350 sec/step)
INFO:tensorflow:global step 255: loss = 2.8778 (0.811 sec/step)
INFO:tensorflow:global step 256: loss = 2.8913 (1.058 sec/step)
INFO:tensorflow:global step 257: loss = 2.8439 (1.010 sec/step)
INFO:tensorflow:global step 258: loss = 2.8722 (0.670 sec/step)
INFO:tensorflow:global step 259: loss = 2.8556 (0.972 sec/step)
INFO:tensorflow:global step 260: loss = 4.0151 (0.821 sec/step)
INFO:tensorflow:global step 261: loss = 2.8105 (1.459 sec/step)
INFO:tensorflow:global step 262: loss = 2.8523 (1.075 sec/step)
INFO:tensorflow:global step 263: loss = 2.8171 (0.908 sec/step)
INFO:tensorflow:global step 264: loss = 2.8303 (1.140 sec/step)
INFO:tensorflow:global step 265: loss = 2.8168 (1.105 sec/step)
INFO:tensorflow:global step 266: loss = 2.7888 (0.975 sec/step)
INFO:tensorflow:global step 267: loss = 

INFO:tensorflow:global step 381: loss = 2.0061 (1.001 sec/step)
INFO:tensorflow:global step 382: loss = 2.0672 (1.087 sec/step)
INFO:tensorflow:global step 383: loss = 2.0735 (0.957 sec/step)
INFO:tensorflow:global step 384: loss = 2.1964 (0.851 sec/step)
INFO:tensorflow:global step 385: loss = 2.0978 (0.999 sec/step)
INFO:tensorflow:global step 386: loss = 2.1018 (1.152 sec/step)
INFO:tensorflow:global step 387: loss = 4.2073 (0.802 sec/step)
INFO:tensorflow:global step 388: loss = 2.0706 (1.292 sec/step)
INFO:tensorflow:global step 389: loss = 2.1571 (0.958 sec/step)
INFO:tensorflow:global step 390: loss = 2.1890 (0.762 sec/step)
INFO:tensorflow:global step 391: loss = 2.1591 (1.251 sec/step)
INFO:tensorflow:global step 392: loss = 1.9400 (1.044 sec/step)
INFO:tensorflow:global step 393: loss = 2.1287 (0.940 sec/step)
INFO:tensorflow:global step 394: loss = 1.9824 (1.188 sec/step)
INFO:tensorflow:global step 395: loss = 2.2570 (0.678 sec/step)
INFO:tensorflow:global step 396: loss = 

INFO:tensorflow:global step 510: loss = 1.5608 (1.037 sec/step)
INFO:tensorflow:global step 511: loss = 1.5225 (1.050 sec/step)
INFO:tensorflow:global step 512: loss = 1.5698 (0.696 sec/step)
INFO:tensorflow:global step 513: loss = 1.5456 (0.792 sec/step)
INFO:tensorflow:global step 514: loss = 1.6174 (1.022 sec/step)
INFO:tensorflow:global step 515: loss = 1.5457 (1.212 sec/step)
INFO:tensorflow:global step 516: loss = 1.6376 (1.078 sec/step)
INFO:tensorflow:global step 517: loss = 1.4887 (0.979 sec/step)
INFO:tensorflow:global step 518: loss = 1.4508 (0.995 sec/step)
INFO:tensorflow:global step 519: loss = 1.6793 (1.133 sec/step)
INFO:tensorflow:global step 520: loss = 1.6558 (1.024 sec/step)
INFO:tensorflow:global step 521: loss = 1.5747 (1.083 sec/step)
INFO:tensorflow:global step 522: loss = 1.6180 (1.098 sec/step)
INFO:tensorflow:global step 523: loss = 1.4597 (0.699 sec/step)
INFO:tensorflow:global step 524: loss = 1.6544 (1.253 sec/step)
INFO:tensorflow:global step 525: loss = 

...

INFO:tensorflow:global step 1399: loss = 1.2420 (0.873 sec/step)
INFO:tensorflow:global step 1400: loss = 1.0249 (1.173 sec/step)
INFO:tensorflow:global step 1401: loss = 1.1491 (0.973 sec/step)
INFO:tensorflow:global step 1402: loss = 1.0104 (0.889 sec/step)
INFO:tensorflow:global step 1403: loss = 1.0572 (1.253 sec/step)
INFO:tensorflow:global step 1404: loss = 1.0805 (0.799 sec/step)
INFO:tensorflow:global step 1405: loss = 2.7451 (1.053 sec/step)
INFO:tensorflow:global step 1406: loss = 1.2178 (1.007 sec/step)
INFO:tensorflow:global step 1407: loss = 1.0403 (1.142 sec/step)
INFO:tensorflow:global step 1408: loss = 1.0225 (0.831 sec/step)
INFO:tensorflow:global step 1409: loss = 1.3529 (0.796 sec/step)
INFO:tensorflow:global step 1410: loss = 1.1454 (1.043 sec/step)
INFO:tensorflow:global step 1411: loss = 1.1968 (1.457 sec/step)
INFO:tensorflow:global step 1412: loss = 1.0568 (0.967 sec/step)
INFO:tensorflow:global step 1413: loss = 1.1467 (1.060 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1526: loss = 0.7799 (0.716 sec/step)
INFO:tensorflow:global step 1527: loss = 0.8788 (1.287 sec/step)
INFO:tensorflow:global step 1528: loss = 0.9703 (1.033 sec/step)
INFO:tensorflow:global step 1529: loss = 0.9769 (1.125 sec/step)
INFO:tensorflow:global step 1530: loss = 0.8026 (1.208 sec/step)
INFO:tensorflow:global step 1531: loss = 0.9538 (1.007 sec/step)
INFO:tensorflow:global step 1532: loss = 0.7720 (0.997 sec/step)
INFO:tensorflow:global step 1533: loss = 0.8626 (1.045 sec/step)
INFO:tensorflow:global step 1534: loss = 0.9809 (1.449 sec/step)
INFO:tensorflow:global step 1535: loss = 0.7943 (0.653 sec/step)
INFO:tensorflow:global step 1536: loss = 0.9559 (0.913 sec/step)
INFO:tensorflow:global step 1537: loss = 0.7793 (0.858 sec/step)
INFO:tensorflow:global step 1538: loss = 0.7667 (0.872 sec/step)
INFO:tensorflow:global step 1539: loss = 0.7616 (1.481 sec/step)
INFO:tensorflow:global step 1540: loss = 0.7899 (1.027 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1653: loss = 0.6489 (0.737 sec/step)
INFO:tensorflow:global step 1654: loss = 1.0921 (0.944 sec/step)
INFO:tensorflow:global step 1655: loss = 0.6917 (0.832 sec/step)
INFO:tensorflow:global step 1656: loss = 0.7395 (1.216 sec/step)
INFO:tensorflow:global step 1657: loss = 0.8806 (1.100 sec/step)
INFO:tensorflow:global step 1658: loss = 0.6579 (0.902 sec/step)
INFO:tensorflow:global step 1659: loss = 0.9276 (1.233 sec/step)
INFO:tensorflow:global step 1660: loss = 0.6857 (1.012 sec/step)
INFO:tensorflow:global step 1661: loss = 0.9115 (1.106 sec/step)
INFO:tensorflow:global step 1662: loss = 0.6640 (1.048 sec/step)
INFO:tensorflow:global step 1663: loss = 0.7759 (0.945 sec/step)
INFO:tensorflow:global step 1664: loss = 0.7745 (1.087 sec/step)
INFO:tensorflow:global step 1665: loss = 0.8118 (1.050 sec/step)
INFO:tensorflow:global step 1666: loss = 0.7727 (1.045 sec/step)
INFO:tensorflow:global step 1667: loss = 0.8127 (0.831 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1777: loss = 0.6624 (0.987 sec/step)
INFO:tensorflow:global step 1778: loss = 0.5541 (1.028 sec/step)
INFO:tensorflow:global step 1779: loss = 0.6105 (1.056 sec/step)
INFO:tensorflow:global step 1780: loss = 0.8385 (0.820 sec/step)
INFO:tensorflow:global step 1781: loss = 0.7469 (1.012 sec/step)
INFO:tensorflow:global step 1782: loss = 0.5932 (1.003 sec/step)
INFO:tensorflow:global step 1783: loss = 0.6856 (1.002 sec/step)
INFO:tensorflow:global step 1784: loss = 0.8269 (1.053 sec/step)
INFO:tensorflow:global step 1785: loss = 0.6226 (1.186 sec/step)
INFO:tensorflow:global step 1786: loss = 0.6095 (0.879 sec/step)
INFO:tensorflow:global step 1787: loss = 0.7791 (0.810 sec/step)
INFO:tensorflow:global step 1788: loss = 0.6846 (1.214 sec/step)
INFO:tensorflow:global step 1789: loss = 0.5816 (1.082 sec/step)
INFO:tensorflow:global step 1790: loss = 0.7283 (0.873 sec/step)
INFO:tensorflow:global step 1791: loss = 0.5480 (1.160 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1904: loss = 0.4539 (0.996 sec/step)
INFO:tensorflow:global step 1905: loss = 0.4463 (0.769 sec/step)
INFO:tensorflow:global step 1906: loss = 0.6739 (1.054 sec/step)
INFO:tensorflow:global step 1907: loss = 0.6188 (1.291 sec/step)
INFO:tensorflow:global step 1908: loss = 1.2133 (0.997 sec/step)
INFO:tensorflow:global step 1909: loss = 0.5286 (0.837 sec/step)
INFO:tensorflow:global step 1910: loss = 0.8006 (1.249 sec/step)
INFO:tensorflow:global step 1911: loss = 0.6629 (0.901 sec/step)
INFO:tensorflow:global step 1912: loss = 0.5219 (1.062 sec/step)
INFO:tensorflow:global step 1913: loss = 0.4729 (0.816 sec/step)
INFO:tensorflow:global step 1914: loss = 0.9698 (1.110 sec/step)
INFO:tensorflow:global step 1915: loss = 0.6207 (1.017 sec/step)
INFO:tensorflow:global step 1916: loss = 0.6590 (0.900 sec/step)
INFO:tensorflow:global step 1917: loss = 0.7917 (1.023 sec/step)
INFO:tensorflow:global step 1918: loss = 0.7002 (0.827 sec/step)
INFO:tensorflow:global st

In [ ]:
import numpy as np
import tensorflow as tf
from datasets import insulators
from nets import inception

from tensorflow.contrib import slim

insulators_data_dir = '/workspace/JiaXuejian'
image_size = resnet_v2.resnet_v2_50.default_image_size
batch_size = 3

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = insulators.get_split('train', insulators_data_dir)
    images, images_raw, labels = load_batch(dataset, height=image_size, width=image_size)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(resnet_v2.resnet_arg_scope()):
        logits, _ = resnet_v2.resnet_v2_50(images, num_classes=dataset.num_classes, is_training=True)

    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
            np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])
    
            for i in range(batch_size): 
                image = np_images_raw[i, :, :, :]
                true_label = np_labels[i]
                predicted_label = np.argmax(np_probabilities[i, :])
                predicted_name = dataset.labels_to_names[predicted_label]
                true_name = dataset.labels_to_names[true_label]
                
                plt.figure()
                plt.imshow(image.astype(np.uint8))
                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
                plt.axis('off')
                plt.show()

In [4]:
import os
from datasets import insulators
from nets import resnet_v2
insulators_data_dir = '/workspace/JiaXuejian'
image_size = resnet_v2.resnet_v2_50.default_image_size
with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = insulators.get_split('train', insulators_data_dir)
    images, _, labels = load_batch(dataset, height=image_size, width=image_size)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(resnet_v2.resnet_arg_scope()):
        logits, end_points = resnet_v2.resnet_v2_50(images, num_classes=dataset.num_classes, is_training=True)
        
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
INFO:tensorflow:Summary name losses/Total Loss is illegal; using losses/Total_Loss instead.


#### 查看网络结构及命名空间

In [5]:
for k, v in end_points.items():
    print('name = {}, shape = {}'.format(v.name, v.get_shape()))


name = resnet_v2_50/conv1/BiasAdd:0, shape = (32, 112, 112, 64)
name = resnet_v2_50/block1/unit_1/bottleneck_v2/shortcut/BiasAdd:0, shape = (32, 56, 56, 256)
name = resnet_v2_50/block1/unit_1/bottleneck_v2/conv1/Relu:0, shape = (32, 56, 56, 64)
name = resnet_v2_50/block1/unit_1/bottleneck_v2/conv2/Relu:0, shape = (32, 56, 56, 64)
name = resnet_v2_50/block1/unit_1/bottleneck_v2/conv3/BiasAdd:0, shape = (32, 56, 56, 256)
name = resnet_v2_50/block1/unit_1/bottleneck_v2/add:0, shape = (32, 56, 56, 256)
name = resnet_v2_50/block1/unit_2/bottleneck_v2/conv1/Relu:0, shape = (32, 56, 56, 64)
name = resnet_v2_50/block1/unit_2/bottleneck_v2/conv2/Relu:0, shape = (32, 56, 56, 64)
name = resnet_v2_50/block1/unit_2/bottleneck_v2/conv3/BiasAdd:0, shape = (32, 56, 56, 256)
name = resnet_v2_50/block1/unit_2/bottleneck_v2/add:0, shape = (32, 56, 56, 256)
name = resnet_v2_50/block1/unit_3/bottleneck_v2/conv1/Relu:0, shape = (32, 56, 56, 64)
name = resnet_v2_50/block1/unit_3/bottleneck_v2/conv2/Relu:0, s